In [91]:
import pandas as pd
import numpy as np
import csv
import datetime
import koleksyon.mcmc as mcmc

# Generative Processes
A generative process, as the name suggests, is a process that generates data.  Most of the time we don't know the true process that generated the data.  That true process often contains human decision making, automation that must be treated as a black box, and other natural and unwieldy phenomenon, that are practically impossible to replicate in the computer.  Simulation is the art and science of replicating these processes as close as practical IN THE COMPUTER.  Simulation is helpful because a simulator that accurately creates data with the same properties of the original process can be used in prediction or hypotheticals analysis to overcome common operations difficulties.

In this example, we have a dataset obtained from Wikipedia that represents raw page hits for the top 10 most visited artist pages on Wikipedia.

In [102]:
df = pd.read_csv("../../data/artist_wiki_page_views-20200101-20201231.csv")
#just filter down for unit tests...
#c1 = df['Date'].str.contains("2020-01-15")
#c2 = df['Date'].str.contains("2020-02-15")
#c3 = df['Date'].str.contains("2020-03-15")
#df[c1 | c2 | c3]
df

,Date,Michael Jackson,Lady Gaga,Eminem,Justin Bieber,Freddie Mercury,Miley Cyrus,Taylor Swift,Lil Wayne,Selena Gomez,Rihanna
0,2020-01-01,21038,22389,13759,23381,56040,14637,16590,7227,12438,12910
1,2020-01-02,20329,15388,13410,17343,30616,13740,16348,6685,14066,11768
2,2020-01-03,17898,12594,13261,27848,26525,13022,15114,7084,13743,11642
3,2020-01-04,19730,15138,12654,26521,32327,12787,15510,6752,14289,10713
4,2020-01-05,18258,16869,12464,26873,33256,13134,13930,6005,12876,12117
...,...,...,...,...,...,...,...,...,...,...,...
361,2020-12-27,31174,17420,20628,15948,30931,15313,23266,5693,12686,14938
362,2020-12-28,28684,15396,22890,17813,20466,14838,22045,5830,12818,12928
363,2020-12-29,26743,13908,22566,18367,18450,14677,20761,5977,12168,13051
364,2020-12-30,23520,13250,20690,17613,18447,14397,19691,5662,12326,14758


Notice how the structure of this data is NOT consistent with most generative processes such as server logs, call logs, encounters, usage logs, sales receipts, and so on.  Its pure summary data.  Most raw datasets have a form that represents a generative process, but many of these datasets are not shared because of the confidential nature, the business value, and the large size.  In this case, we will make up a dataset that has the correct form based on a 'made up' process.  Data Scientists in industry should notice immediately a similarity between this dataset and datasets we commonly encounter.

The first step in making our dataset is to extract album prices from a well known retailer (Dec 2020):

In [74]:
#Artist-Album costs (0.0 represents that you don't buy and albumn, .99 represents you buy just a song)
MichaelJackson = np.array([0.0,0.99,8.64,8.69,12.33,12.96,38.99,30.12,13.99,17.25])
LadyGaga = np.array([0.0,0.99,14.28,11.20,11.25,14.98,13.69,9.99,18.95])
Eminem = np.array([0.0,0.99,15.99,9.33,21.61,22.37,12.80,10.75,11.70])
JustinBieber = np.array([0.0,0.99,10.70,9.49,14.65,29.18,21.93,15.95,19.90,37.98])
FreddieMercury = np.array([0.0,0.99,14.74,11.50,18.99,12.49,14.54,10.99,11.89,16.53,11.70,9.71,12.39])
MileyCyrus = np.array([0.0,0.99,11.18,6.98,9.21,9.95,9.49])
TaylorSwift = np.array([0.0,0.99,13.98,16.99,13.51,8.97,15.02,7.00,13.97,8.97,6.86])
LilWayne = np.array([0.0,0.99,11.55,16.00,29.47,13.41,9.68,15.95,11.99,16.63])
SelenaGomez = np.array([0.0,0.99,12.59,10.91,36.57,16.52])
Rihanna = np.array([0.0,0.99,13.98,10.25,22.90,6.32,9.19])
ArtistAlbums = {}
ArtistAlbums["Michael Jackson"] = MichaelJackson
ArtistAlbums["Lady Gaga"] = LadyGaga
ArtistAlbums["Eminem"] = Eminem
ArtistAlbums["Justin Bieber"] = JustinBieber
ArtistAlbums["Freddie Mercury"] = FreddieMercury
ArtistAlbums["Miley Cyrus"] = MileyCyrus
ArtistAlbums["Taylor Swift"] = TaylorSwift
ArtistAlbums["Lil Wayne"] = LilWayne
ArtistAlbums["Selena Gomez"] = SelenaGomez
ArtistAlbums["Rihanna"] = Rihanna
             

For this example, we will assume that people check Wikipedia for one of the above artists, and then a certain percent of them (1%) order music from our retailer.  A careful look at the data shows that there are 'influencer' spikes that likely go with a new album release, a news story, a tour or other activity that generates 'buzz'.   We don't know what the price of new products, or when the price of existing processes are going to change.  We also don't know without real logs the overall interest in a given albumn over time.  All of these different variables are examples of 'black box' variables that we wish to be able to simulate.  The fact that so many of these variables are so difficult to pin down is really the point!

For the current task, creating a dataset with approximately the correct structure, we will assume that 30% of visits result in a purchase, and of those, 80% just buy a single song (0.99 cents).  The final 20% of purchases is distributed equally between the existing albums.

In [75]:
purchase_probability = {}
for k,v in ArtistAlbums.items():
    print(k)
    print(v)
    proba = []
    proba.append(0.70)  #30% purchases, 70% not purchases
    proba.append(0.24)
    r =  0.06 / (len(v) - 2)
    for i in range(0,len(v)-2):
        proba.append(r)
    proba = np.array(proba)
    print(proba)
    purchase_probability[k] = proba

Michael Jackson
[ 0.    0.99  8.64  8.69 12.33 12.96 38.99 30.12 13.99 17.25]
[0.7    0.24   0.0075 0.0075 0.0075 0.0075 0.0075 0.0075 0.0075 0.0075]
Lady Gaga
[ 0.    0.99 14.28 11.2  11.25 14.98 13.69  9.99 18.95]
[0.7        0.24       0.00857143 0.00857143 0.00857143 0.00857143
 0.00857143 0.00857143 0.00857143]
Eminem
[ 0.    0.99 15.99  9.33 21.61 22.37 12.8  10.75 11.7 ]
[0.7        0.24       0.00857143 0.00857143 0.00857143 0.00857143
 0.00857143 0.00857143 0.00857143]
Justin Bieber
[ 0.    0.99 10.7   9.49 14.65 29.18 21.93 15.95 19.9  37.98]
[0.7    0.24   0.0075 0.0075 0.0075 0.0075 0.0075 0.0075 0.0075 0.0075]
Freddie Mercury
[ 0.    0.99 14.74 11.5  18.99 12.49 14.54 10.99 11.89 16.53 11.7   9.71
 12.39]
[0.7        0.24       0.00545455 0.00545455 0.00545455 0.00545455
 0.00545455 0.00545455 0.00545455 0.00545455 0.00545455 0.00545455
 0.00545455]
Miley Cyrus
[ 0.    0.99 11.18  6.98  9.21  9.95  9.49]
[0.7   0.24  0.012 0.012 0.012 0.012 0.012]
Taylor Swift
[ 0.    0.99

In [77]:
#generate a dataframe shaped like a purchase log for a single cell in the page view table
def generate_amounts(artist, date, count):
    amounts = np.random.choice(ArtistAlbums[artist], count, p=purchase_probability[artist])
    df = pd.DataFrame(data=amounts,  columns=["amount"])
    df['artist'] = artist 
    df['date'] = date
    return df


In [44]:
dflog = pd.DataFrame(columns=['date', 'artist', 'amount'])
dflog = dflog.append({'date':"1-1-01",'artist':'MJ','amount':21.33}, ignore_index=True)
dflog

,date,artist,amount
0,1-1-01,MJ,21.33


In [88]:
scale_factor = 1000 #if scale factor == 1, dataset will be true size, 10==one tenth the size, 100=one one hundreth ect.
def generate_log_data(df_page_view_counts):
    #iterate over the page view counts
    dflog = pd.DataFrame(columns=['date', 'artist', 'amount']) #dataframe to place the log
    rws, clms = df_page_view_counts.shape
    date_time = datetime.datetime.now()
    for i in range(rws):
        for col in list(df_page_view_counts.columns):
            if 'Date' in col:
                date_time = datetime.datetime.strptime(df_page_view_counts.at[i,col], '%Y-%m-%d')
                print(date_time)
            else:
                #print(df.iat[i,j])
                artist = col
                visits = int(df_page_view_counts.at[i,col])
                dfl = generate_amounts(artist, date_time, visits // scale_factor)
                print(dfl)
                dflog = dflog.append(dfl, ignore_index=True)
                print(len(dflog))
    dflog['date'] = pd.to_datetime(dflog['date'])
    return dflog

dflog = generate_log_data(df)

2020-01-01 00:00:00
    amount           artist       date
0     0.00  Michael Jackson 2020-01-01
1     0.99  Michael Jackson 2020-01-01
2     0.00  Michael Jackson 2020-01-01
3     0.00  Michael Jackson 2020-01-01
4     0.00  Michael Jackson 2020-01-01
5     0.00  Michael Jackson 2020-01-01
6     0.00  Michael Jackson 2020-01-01
7    12.96  Michael Jackson 2020-01-01
8     0.00  Michael Jackson 2020-01-01
9     0.99  Michael Jackson 2020-01-01
10    0.00  Michael Jackson 2020-01-01
11    0.99  Michael Jackson 2020-01-01
12    0.00  Michael Jackson 2020-01-01
13   12.96  Michael Jackson 2020-01-01
14    0.99  Michael Jackson 2020-01-01
15    0.00  Michael Jackson 2020-01-01
16    0.00  Michael Jackson 2020-01-01
17    0.00  Michael Jackson 2020-01-01
18    0.00  Michael Jackson 2020-01-01
19    0.99  Michael Jackson 2020-01-01
20    0.00  Michael Jackson 2020-01-01
21
    amount     artist       date
0     0.00  Lady Gaga 2020-01-01
1     0.00  Lady Gaga 2020-01-01
2     0.00  Lady Gag

684
    amount     artist       date
0     0.99  Lady Gaga 2020-01-05
1     0.00  Lady Gaga 2020-01-05
2     0.99  Lady Gaga 2020-01-05
3     0.00  Lady Gaga 2020-01-05
4     0.00  Lady Gaga 2020-01-05
5     0.00  Lady Gaga 2020-01-05
6    18.95  Lady Gaga 2020-01-05
7     0.99  Lady Gaga 2020-01-05
8     0.00  Lady Gaga 2020-01-05
9     0.99  Lady Gaga 2020-01-05
10    0.00  Lady Gaga 2020-01-05
11    0.00  Lady Gaga 2020-01-05
12    0.00  Lady Gaga 2020-01-05
13    0.00  Lady Gaga 2020-01-05
14    0.00  Lady Gaga 2020-01-05
15    0.99  Lady Gaga 2020-01-05
700
    amount  artist       date
0     0.00  Eminem 2020-01-05
1     0.00  Eminem 2020-01-05
2     0.00  Eminem 2020-01-05
3     0.00  Eminem 2020-01-05
4     0.00  Eminem 2020-01-05
5     0.00  Eminem 2020-01-05
6     0.00  Eminem 2020-01-05
7     0.00  Eminem 2020-01-05
8     0.00  Eminem 2020-01-05
9     0.00  Eminem 2020-01-05
10    0.00  Eminem 2020-01-05
11    0.99  Eminem 2020-01-05
712
    amount         artist       date


1390
    amount     artist       date
0     0.00  Lady Gaga 2020-01-10
1     0.99  Lady Gaga 2020-01-10
2     0.00  Lady Gaga 2020-01-10
3     0.00  Lady Gaga 2020-01-10
4     0.00  Lady Gaga 2020-01-10
5     0.99  Lady Gaga 2020-01-10
6     0.00  Lady Gaga 2020-01-10
7     0.00  Lady Gaga 2020-01-10
8     0.00  Lady Gaga 2020-01-10
9     0.99  Lady Gaga 2020-01-10
10    0.99  Lady Gaga 2020-01-10
1401
    amount  artist       date
0     0.00  Eminem 2020-01-10
1     0.99  Eminem 2020-01-10
2     0.99  Eminem 2020-01-10
3     0.00  Eminem 2020-01-10
4     0.00  Eminem 2020-01-10
5     0.00  Eminem 2020-01-10
6     0.00  Eminem 2020-01-10
7     0.99  Eminem 2020-01-10
8     0.99  Eminem 2020-01-10
9     0.00  Eminem 2020-01-10
10    0.99  Eminem 2020-01-10
11    0.99  Eminem 2020-01-10
1413
    amount         artist       date
0     9.49  Justin Bieber 2020-01-10
1     0.00  Justin Bieber 2020-01-10
2     0.00  Justin Bieber 2020-01-10
3     0.00  Justin Bieber 2020-01-10
4     0.99  Ju

13    0.99  Taylor Swift 2020-01-14
2099
    amount     artist       date
0     0.00  Lil Wayne 2020-01-14
1     0.99  Lil Wayne 2020-01-14
2     0.00  Lil Wayne 2020-01-14
3     0.99  Lil Wayne 2020-01-14
4     0.00  Lil Wayne 2020-01-14
5     0.00  Lil Wayne 2020-01-14
6     0.00  Lil Wayne 2020-01-14
7     0.99  Lil Wayne 2020-01-14
8     0.99  Lil Wayne 2020-01-14
9     0.99  Lil Wayne 2020-01-14
10   16.63  Lil Wayne 2020-01-14
2110
    amount        artist       date
0     0.00  Selena Gomez 2020-01-14
1     0.00  Selena Gomez 2020-01-14
2     0.00  Selena Gomez 2020-01-14
3     0.00  Selena Gomez 2020-01-14
4     0.00  Selena Gomez 2020-01-14
5     0.00  Selena Gomez 2020-01-14
6     0.00  Selena Gomez 2020-01-14
7     0.00  Selena Gomez 2020-01-14
8     0.99  Selena Gomez 2020-01-14
9     0.99  Selena Gomez 2020-01-14
10   10.91  Selena Gomez 2020-01-14
11    0.00  Selena Gomez 2020-01-14
12    0.00  Selena Gomez 2020-01-14
13    0.00  Selena Gomez 2020-01-14
14    0.00  Selena

2957
    amount     artist       date
0     0.00  Lady Gaga 2020-01-19
1     0.99  Lady Gaga 2020-01-19
2     0.00  Lady Gaga 2020-01-19
3     0.00  Lady Gaga 2020-01-19
4     0.99  Lady Gaga 2020-01-19
5     0.00  Lady Gaga 2020-01-19
6     0.00  Lady Gaga 2020-01-19
7     0.00  Lady Gaga 2020-01-19
8     0.99  Lady Gaga 2020-01-19
9     0.00  Lady Gaga 2020-01-19
10    0.99  Lady Gaga 2020-01-19
11    0.00  Lady Gaga 2020-01-19
2969
    amount  artist       date
0    21.61  Eminem 2020-01-19
1     0.00  Eminem 2020-01-19
2     0.00  Eminem 2020-01-19
3     0.99  Eminem 2020-01-19
4     0.99  Eminem 2020-01-19
5     0.00  Eminem 2020-01-19
6    15.99  Eminem 2020-01-19
7     0.99  Eminem 2020-01-19
8     0.99  Eminem 2020-01-19
9     0.00  Eminem 2020-01-19
10    0.00  Eminem 2020-01-19
11    0.00  Eminem 2020-01-19
12    0.00  Eminem 2020-01-19
13    0.00  Eminem 2020-01-19
14    0.99  Eminem 2020-01-19
15    0.00  Eminem 2020-01-19
16    0.99  Eminem 2020-01-19
17    0.00  Eminem 20

...

54053
    amount  artist       date
0     0.00  Eminem 2020-12-13
1     0.99  Eminem 2020-12-13
2     0.99  Eminem 2020-12-13
3     0.00  Eminem 2020-12-13
4     0.00  Eminem 2020-12-13
5     0.99  Eminem 2020-12-13
6     0.00  Eminem 2020-12-13
7     0.00  Eminem 2020-12-13
8     0.00  Eminem 2020-12-13
9     0.00  Eminem 2020-12-13
10    0.00  Eminem 2020-12-13
11   10.75  Eminem 2020-12-13
12    0.00  Eminem 2020-12-13
13    0.99  Eminem 2020-12-13
14    0.00  Eminem 2020-12-13
15    0.00  Eminem 2020-12-13
54069
    amount         artist       date
0     0.99  Justin Bieber 2020-12-13
1     0.00  Justin Bieber 2020-12-13
2     0.99  Justin Bieber 2020-12-13
3     0.00  Justin Bieber 2020-12-13
4     0.00  Justin Bieber 2020-12-13
5     0.00  Justin Bieber 2020-12-13
6     0.00  Justin Bieber 2020-12-13
7     0.00  Justin Bieber 2020-12-13
8    29.18  Justin Bieber 2020-12-13
9    37.98  Justin Bieber 2020-12-13
10    0.99  Justin Bieber 2020-12-13
11    0.00  Justin Bieber 2020-12-

54780
    amount     artist       date
0     0.00  Lady Gaga 2020-12-17
1     0.00  Lady Gaga 2020-12-17
2     0.99  Lady Gaga 2020-12-17
3     0.00  Lady Gaga 2020-12-17
4     0.00  Lady Gaga 2020-12-17
5     0.00  Lady Gaga 2020-12-17
6     0.00  Lady Gaga 2020-12-17
7     0.00  Lady Gaga 2020-12-17
8     0.00  Lady Gaga 2020-12-17
9     0.00  Lady Gaga 2020-12-17
10    0.99  Lady Gaga 2020-12-17
11    0.00  Lady Gaga 2020-12-17
12    0.00  Lady Gaga 2020-12-17
13    0.00  Lady Gaga 2020-12-17
54794
    amount  artist       date
0     0.00  Eminem 2020-12-17
1     0.99  Eminem 2020-12-17
2     0.00  Eminem 2020-12-17
3     0.00  Eminem 2020-12-17
4     0.00  Eminem 2020-12-17
5     0.00  Eminem 2020-12-17
6     0.00  Eminem 2020-12-17
7     0.00  Eminem 2020-12-17
8     0.00  Eminem 2020-12-17
9     0.99  Eminem 2020-12-17
10    0.00  Eminem 2020-12-17
11    0.00  Eminem 2020-12-17
12    0.00  Eminem 2020-12-17
13   11.70  Eminem 2020-12-17
14    0.99  Eminem 2020-12-17
54809
    amo

55516
    amount     artist       date
0     0.00  Lady Gaga 2020-12-21
1     0.99  Lady Gaga 2020-12-21
2     0.00  Lady Gaga 2020-12-21
3     0.00  Lady Gaga 2020-12-21
4     0.00  Lady Gaga 2020-12-21
5     0.00  Lady Gaga 2020-12-21
6     0.00  Lady Gaga 2020-12-21
7     0.00  Lady Gaga 2020-12-21
8     0.00  Lady Gaga 2020-12-21
9     0.99  Lady Gaga 2020-12-21
10    0.00  Lady Gaga 2020-12-21
11    0.00  Lady Gaga 2020-12-21
12    0.00  Lady Gaga 2020-12-21
13    0.00  Lady Gaga 2020-12-21
55530
    amount  artist       date
0     0.00  Eminem 2020-12-21
1     0.99  Eminem 2020-12-21
2     0.00  Eminem 2020-12-21
3     0.00  Eminem 2020-12-21
4     0.00  Eminem 2020-12-21
5     9.33  Eminem 2020-12-21
6     0.00  Eminem 2020-12-21
7     0.00  Eminem 2020-12-21
8     0.00  Eminem 2020-12-21
9     0.00  Eminem 2020-12-21
10    0.00  Eminem 2020-12-21
11    0.99  Eminem 2020-12-21
12    0.99  Eminem 2020-12-21
13    0.00  Eminem 2020-12-21
14    0.00  Eminem 2020-12-21
15    0.00  E

56180
    amount  artist       date
0     0.00  Eminem 2020-12-25
1    21.61  Eminem 2020-12-25
2     0.00  Eminem 2020-12-25
3     0.00  Eminem 2020-12-25
4     0.00  Eminem 2020-12-25
5     0.00  Eminem 2020-12-25
6     0.00  Eminem 2020-12-25
7     0.99  Eminem 2020-12-25
8     0.00  Eminem 2020-12-25
9     0.00  Eminem 2020-12-25
10    0.00  Eminem 2020-12-25
11    0.00  Eminem 2020-12-25
12    0.00  Eminem 2020-12-25
13    0.99  Eminem 2020-12-25
14    0.99  Eminem 2020-12-25
15    0.00  Eminem 2020-12-25
16    0.00  Eminem 2020-12-25
17    0.99  Eminem 2020-12-25
18    0.00  Eminem 2020-12-25
19   12.80  Eminem 2020-12-25
56200
    amount         artist       date
0     0.00  Justin Bieber 2020-12-25
1     0.99  Justin Bieber 2020-12-25
2     0.00  Justin Bieber 2020-12-25
3     0.99  Justin Bieber 2020-12-25
4     0.99  Justin Bieber 2020-12-25
5     0.00  Justin Bieber 2020-12-25
6     0.99  Justin Bieber 2020-12-25
7     0.99  Justin Bieber 2020-12-25
8     0.00  Justin Bieber

25    0.00  Michael Jackson 2020-12-29
56912
    amount     artist       date
0     0.00  Lady Gaga 2020-12-29
1     0.99  Lady Gaga 2020-12-29
2     0.00  Lady Gaga 2020-12-29
3     0.00  Lady Gaga 2020-12-29
4     0.00  Lady Gaga 2020-12-29
5     0.00  Lady Gaga 2020-12-29
6     0.00  Lady Gaga 2020-12-29
7     0.00  Lady Gaga 2020-12-29
8     0.99  Lady Gaga 2020-12-29
9    11.25  Lady Gaga 2020-12-29
10    0.00  Lady Gaga 2020-12-29
11    0.99  Lady Gaga 2020-12-29
12    0.99  Lady Gaga 2020-12-29
56925
    amount  artist       date
0     0.99  Eminem 2020-12-29
1     0.00  Eminem 2020-12-29
2     0.00  Eminem 2020-12-29
3    11.70  Eminem 2020-12-29
4     0.99  Eminem 2020-12-29
5     0.99  Eminem 2020-12-29
6     0.99  Eminem 2020-12-29
7     0.99  Eminem 2020-12-29
8     0.00  Eminem 2020-12-29
9     0.99  Eminem 2020-12-29
10    0.00  Eminem 2020-12-29
11    0.99  Eminem 2020-12-29
12    0.00  Eminem 2020-12-29
13    0.00  Eminem 2020-12-29
14    0.00  Eminem 2020-12-29
15    0

In [89]:
dflog

,date,artist,amount
0,2020-01-01,Michael Jackson,0.00
1,2020-01-01,Michael Jackson,0.99
2,2020-01-01,Michael Jackson,0.00
3,2020-01-01,Michael Jackson,0.00
4,2020-01-01,Michael Jackson,0.00
...,...,...,...
57385,2020-12-31,Rihanna,0.00
57386,2020-12-31,Rihanna,0.00
57387,2020-12-31,Rihanna,0.00
57388,2020-12-31,Rihanna,0.00


In [93]:
n_sims = 10
simulator = mcmc.SimulationArray(dflog, 'artist', 'date', 'amount', n_sims)

Preprocessing Data Frame Into Simulation Array...
0


KeyError: Timestamp('2020-01-01 00:00:00')

In [ ]:
TODO: take the example from Finance and finish this example!